<a href="https://colab.research.google.com/github/gitconnoisseur/toxicity-model/blob/master/ecfp_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import autogluon as ag
from tdc.single_pred import Tox
import deepchem
import xgboost

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

Found local copy...


Run benchmark using xgboost with an ECFP featurizer:

In [4]:
predictions_list = []
metric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.mean_absolute_error)
ecfpFeat = deepchem.deepchem.feat.CircularFingerprint(radius=4)
xgb_reg = xgboost.XGBRegressor()

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('LD50_Zhu') 
    
    predictions = {}
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)  

    #featurize training, valid, and test data for xgboost
    ecfp_f_train = ecfpFeat.featurize(train.iloc[:,1].to_list())
    ecfp_f_valid = ecfpFeat.featurize(valid.iloc[:,1].to_list())
    ecfp_f_test = ecfpFeat.featurize(test.iloc[:,1].to_list())
    train_val_f = ecfpFeat.featurize(train_val.iloc[:,1].to_list())

    #ecfp_f_valid.shape = (739, 2048)

    #valid_data = list(zip(ecfp_f_valid, valid.iloc[:,2].to_list()))
    valid_data = [(ecfp_f_train, train.iloc[:,2].to_list()), (ecfp_f_valid, valid.iloc[:,2].to_list())]

    #fit xgboost model and store np ndarray in xgb_pred
    #below you can use ecfp_f_train for X instead and the valid data with eval_set
    xgb_reg.fit(X=train_val_f, y=train_val.iloc[:,2], eval_metric="mae", verbose=True)
    xgb_pred = xgb_reg.predict(X=ecfp_f_test)

    # store test predictions in y_pred_test
    y_pred_test = xgb_pred

    print("predictions for run #")
    print(seed)
    print(y_pred_test[0:5])
        
    predictions[name] = y_pred_test
    predictions_list.append(predictions)

print("Prediction List:")
print(predictions_list)

generating training, validation splits...
100%|██████████| 5907/5907 [00:01<00:00, 3745.55it/s]
generating training, validation splits...


predictions for run #
1
[2.1006067 2.2473924 3.3978975 4.0252185 4.210433 ]


100%|██████████| 5907/5907 [00:01<00:00, 4184.77it/s]
generating training, validation splits...


predictions for run #
2
[2.1006067 2.2473924 3.3978975 4.0252185 4.210433 ]


100%|██████████| 5907/5907 [00:01<00:00, 3561.65it/s]
generating training, validation splits...


predictions for run #
3
[2.1006067 2.2473924 3.3978975 4.0252185 4.210433 ]


100%|██████████| 5907/5907 [00:01<00:00, 4376.26it/s]
generating training, validation splits...


predictions for run #
4
[2.1006067 2.2473924 3.3978975 4.0252185 4.210433 ]


100%|██████████| 5907/5907 [00:01<00:00, 4156.43it/s]


predictions for run #
5
[2.1006067 2.2473924 3.3978975 4.0252185 4.210433 ]
Prediction List:
[{'ld50_zhu': array([2.1006067, 2.2473924, 3.3978975, ..., 2.4514642, 2.0766413,
       2.1703484], dtype=float32)}, {'ld50_zhu': array([2.1006067, 2.2473924, 3.3978975, ..., 2.4514642, 2.0766413,
       2.1703484], dtype=float32)}, {'ld50_zhu': array([2.1006067, 2.2473924, 3.3978975, ..., 2.4514642, 2.0766413,
       2.1703484], dtype=float32)}, {'ld50_zhu': array([2.1006067, 2.2473924, 3.3978975, ..., 2.4514642, 2.0766413,
       2.1703484], dtype=float32)}, {'ld50_zhu': array([2.1006067, 2.2473924, 3.3978975, ..., 2.4514642, 2.0766413,
       2.1703484], dtype=float32)}]


In [5]:
for i in predictions_list:
    print(group.evaluate(i))

group.evaluate_many(predictions_list)

{'ld50_zhu': {'mae': 0.661}}
{'ld50_zhu': {'mae': 0.661}}
{'ld50_zhu': {'mae': 0.661}}
{'ld50_zhu': {'mae': 0.661}}
{'ld50_zhu': {'mae': 0.661}}


{'ld50_zhu': [0.661, 0.0]}

In [14]:
#code below may be used for hyperparam opt as needed

hyperOptMetric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.r2_score)

def model_builder(**model_params):
  return deepchem.deepchem.models.GraphConvModel(n_tasks=1, mode="regression", graph_conv_layers=[128, 128, 128])

#optimizer = deepchem.deepchem.hyper.GaussianProcessHyperparamOpt(model_builder)

optimizer = deepchem.deepchem.hyper.GaussianProcessHyperparamOpt(model_builder)

params_dict = {"dropout": .01, "dense_layer_size": 200}

best_model, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict, train_data, valid_data, hyperOptMetric, search_range=10)

Evaluation 	 Proposed point 	  Current eval. 	 Best eval.
init   	 [9.54472202e-02 1.66300000e+03]. 	  0.05614210575885126 	 0.13022634235269537
init   	 [7.81778992e-02 1.95200000e+03]. 	  0.11803599063732828 	 0.13022634235269537
init   	 [6.78009843e-02 3.88000000e+02]. 	  0.13022634235269537 	 0.13022634235269537
1      	 [1.21594231e-02 4.02000000e+02]. 	  0.10297839213545745 	 0.13022634235269537
2      	 [5.63577392e-02 1.94996902e+03]. 	  0.07543380244710662 	 0.13022634235269537
3      	 [8.73771372e-02 3.99869355e+02]. 	  0.10647494762543674 	 0.13022634235269537
4      	 [5.73742248e-02 3.86042513e+02]. 	  -0.015072240034492701 	 0.13022634235269537
5      	 [1.44460922e-02 1.93100000e+03]. 	  0.04720652864226782 	 0.13022634235269537
6      	 [6.92807233e-02 1.93369230e+03]. 	  0.21513004052653117 	 0.21513004052653117
7      	 [1.00000000e-01 3.89993332e+02]. 	  0.08601514697821022 	 0.21513004052653117
8      	 [5.25671057e-03 1.94747276e+03]. 	  0.09740603500068701 	 0.2

In [15]:
print(best_hyperparams)

{'dropout': 0.0661305398871231, 'dense_layer_size': 397}
